# Import

In [22]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 199)

from datetime import datetime
import re
import os
import os.path
import math
import sys
import csv
import glob
import seaborn as sns
import numpy as np
import shap 
from collections import Counter

# import mlflow
# import mlflow.sklearn
# import pysftp

from sklearn.metrics import confusion_matrix
import seaborn as sns
pd.options.display.float_format = "{:.2f}".format

import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import traceback
import string

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm 

import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, accuracy_score, log_loss, roc_auc_score, f1_score

%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/data_preparation/utils.py"
%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/visualization/common_visualization.py"
%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/data_preparation/common_eda.py"
%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/modelling/regression_model.py"
%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/data_preparation/common_dictionary.py"

print("current working folder: ", os.getcwd())
user_name = "chaupham"
# print("notebook kernel id: ", get_notebook_kernel_id())

current working folder:  /Users/chau/minhchau/jupyter_projects/plant_project


# Helper funcs

In [64]:
def convert_to_up_down_noChange(x, thres):
    if x[c] - x[base_col] > thres:  ##UP
        return "2"
    elif x[c] + thres < x[base_col]:
        return  "1" ## DOWN
    else:
        return "0" # NO_CHANGE

# Read input

In [65]:
df = pd.read_csv("./data/cleaned_data.csv")
df.shape                 

(20450, 14)

In [66]:
df.label.value_counts()

0    19245
1     1205
Name: label, dtype: int64

In [67]:
df.head()

,atID,wthp6,wthp6_2,wtlp6,wtlp6_2,wthp5,wthp5_2,wtlp5,wtlp5_2,wtal,wtal_2,wtfe,wtfe_2,label
0,AT1G01020,18.51,17.60,17.80,20.36,18.99,21.93,18.29,21.48,19.01,17.58,18.21,18.19,0
1,AT1G01030,0.82,1.07,0.68,0.45,0.54,0.40,0.56,0.29,0.22,0.81,0.93,0.62,1
2,AT1G01040,36.57,34.09,31.92,53.51,37.06,34.78,43.44,49.22,42.09,36.11,32.94,17.10,0
3,AT1G01050,103.58,109.89,103.21,96.22,100.88,106.00,97.60,100.95,89.76,92.77,91.97,112.10,0
4,AT1G01060,1.94,1.78,2.93,2.88,3.15,3.67,2.44,2.13,1.34,1.89,3.52,1.89,1


In [68]:
wt_cols = ['wthp6', 'wtlp6',  'wthp5',  'wtlp5',  'wtal',  'wtfe']
wt_AVE_cols = [c + "_ave" for c in wt_cols]
wt_new_cols = wt_AVE_cols[1:]
for c in wt_cols:
    df[c + "_ave"] = (df[c] + df[c + "_2"]) / 2
    
df.columns

Index(['atID', 'wthp6', 'wthp6_2', 'wtlp6', 'wtlp6_2', 'wthp5', 'wthp5_2', 'wtlp5', 'wtlp5_2', 'wtal', 'wtal_2', 'wtfe', 'wtfe_2', 'label', 'wthp6_ave', 'wtlp6_ave', 'wthp5_ave', 'wtlp5_ave', 'wtal_ave', 'wtfe_ave'], dtype='object')

In [69]:
df.head()

,atID,wthp6,wthp6_2,wtlp6,wtlp6_2,wthp5,wthp5_2,wtlp5,wtlp5_2,wtal,wtal_2,wtfe,wtfe_2,label,wthp6_ave,wtlp6_ave,wthp5_ave,wtlp5_ave,wtal_ave,wtfe_ave
0,AT1G01020,18.51,17.60,17.80,20.36,18.99,21.93,18.29,21.48,19.01,17.58,18.21,18.19,0,18.06,19.08,20.46,19.88,18.30,18.20
1,AT1G01030,0.82,1.07,0.68,0.45,0.54,0.40,0.56,0.29,0.22,0.81,0.93,0.62,1,0.94,0.57,0.47,0.43,0.52,0.77
2,AT1G01040,36.57,34.09,31.92,53.51,37.06,34.78,43.44,49.22,42.09,36.11,32.94,17.10,0,35.33,42.71,35.92,46.33,39.10,25.02
3,AT1G01050,103.58,109.89,103.21,96.22,100.88,106.00,97.60,100.95,89.76,92.77,91.97,112.10,0,106.73,99.72,103.44,99.28,91.26,102.04
4,AT1G01060,1.94,1.78,2.93,2.88,3.15,3.67,2.44,2.13,1.34,1.89,3.52,1.89,1,1.86,2.91,3.41,2.29,1.61,2.71


In [70]:
data

,atID,wtlp6_ave,wthp5_ave,wtlp5_ave,wtal_ave,wtfe_ave
0,AT1G01020,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE
1,AT1G01030,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE
2,AT1G01040,NO_CHANGE,NO_CHANGE,UP,NO_CHANGE,DOWN
3,AT1G01050,NO_CHANGE,NO_CHANGE,NO_CHANGE,DOWN,NO_CHANGE
4,AT1G01060,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE
...,...,...,...,...,...,...
20445,ATMG01130,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE
20446,ATMG01200,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE
20447,ATMG01360,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE
20448,ATMG01370,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE,NO_CHANGE


In [76]:
data = df[["atID"]].copy()
thres = 10
base_col = wt_AVE_cols[0]
for c in wt_new_cols:
    data[c] = df.apply(lambda x: convert_to_up_down_noChange(x, thres), 1)
stats_df = data.groupby(wt_new_cols).count().reset_index().sort_values("atID", ascending= False).reset_index(drop=True).rename(columns = {"atID": "#genes"})
stats_df

,wtlp6_ave,wthp5_ave,wtlp5_ave,wtal_ave,wtfe_ave,#genes,group_id
0,0,0,0,0,0,14396,14396
1,0,0,0,0,2,904,904
2,0,0,0,0,1,613,613
3,0,0,0,2,2,333,333
4,1,1,1,1,1,318,318
5,0,0,0,2,0,240,240
6,0,0,0,1,0,228,228
7,2,0,2,0,1,181,181
8,0,0,0,1,1,155,155
9,1,1,1,1,2,147,147


In [78]:
data["group_id"] = data[wt_new_cols].apply(lambda x: "".join(x[c] for c in wt_new_cols),1)

data.head()

,atID,wtlp6_ave,wthp5_ave,wtlp5_ave,wtal_ave,wtfe_ave,group_id
0,AT1G01020,0,0,0,0,0,00000
1,AT1G01030,0,0,0,0,0,00000
2,AT1G01040,0,0,2,0,1,00201
3,AT1G01050,0,0,0,1,0,00010
4,AT1G01060,0,0,0,0,0,00000


In [81]:
unique_groups = data.group_id.unique()
DataFrameDict = {elem : pd.DataFrame for elem in unique_groups}

for key in DataFrameDict.keys():
    DataFrameDict[key] = data[:][data.group_id == key]

,atID,wtlp6_ave,wthp5_ave,wtlp5_ave,wtal_ave,wtfe_ave,group_id
0,AT1G01020,0,0,0,0,0,00000
1,AT1G01030,0,0,0,0,0,00000
4,AT1G01060,0,0,0,0,0,00000
5,AT1G01070,0,0,0,0,0,00000
6,AT1G01080,0,0,0,0,0,00000
...,...,...,...,...,...,...,...
20444,ATMG01120,0,0,0,0,0,00000
20445,ATMG01130,0,0,0,0,0,00000
20446,ATMG01200,0,0,0,0,0,00000
20447,ATMG01360,0,0,0,0,0,00000
